# News Hour - Part II

In this notebook, we handle what is essentially the second half of the setup process. This involves rendering all of our data into a shape suitable for a more explicitly dynamic panel analysis. Basically, it amounts to doing some shifting operations in mata so that our data is set up a bit better than it was before. 

We also go ahead and estimate a dynamic version of the viewership model based on this. Estimation of this model is time-consuming, and to save time and trouble, we use the method described in [this paper](http://papers.ssrn.com/sol3/papers.cfm?abstract_id=420371), which is also described in [this paper](http://www.stata-journal.com/article.html?article=st0354). 

To estimate things using this method, you can read the latter paper above, and should download and install the `AMCMC` `Stata` module from `SSC`. It can be found [here](https://ideas.repec.org/c/boc/bocode/s457613.html).  This also requires installation of the `moremata` module from [here](https://ideas.repec.org/c/boc/bocode/s455001.html).

## Some theoretical preliminaries

As we state in the paper, we rely on what can be called a **Share Form Nested Multinomial Logit**. What is this, and how does it typically work? The basic idea, which originates with [Berry's 1994 paper](http://econpapers.repec.org/article/rjerandje/v_3a25_3ay_3a1994_3ai_3asummer_3ap_3a242-262.htm) is to render what is seemingly a difficult model into a linear form. 

First write viewership shares like this:
$$ 
s_i=s_{i|b}s_b
$$

where $s_b$ in denotes the total viewing share among stations with type $b$ broadcasts, and $s_{i|b}$ denotes the within-group share of station $i$. In a nested multinomial logit model, a substitutability parameter $\mu_b \in [0,1]$ is used to characterize the degree to which items within a given type substitute for one another. Following Berry, define:

$$
D_b=\sum_{k \in S_b} e^{\frac{u_k(b_k)}{1-\mu_b}}
$$

so that we can write:

$$
s_{i|b}=\frac{e^{\frac{u_k(b_k)}{1-\mu_b}}}{D_b}, \quad s_b=\frac{D_b^{1-\mu_b}}{\sum_{k \in B}D_k^{1-\mu_k}}
$$

Berry shows that this can be cast, with a little bit of algebra, into linear form:

$$
y_i=\ln s_i-\ln s_0=u_i(b)+\mu_b \ln s_{i|b}
$$

Berry details how to do this. The big thing to note about this last equation is that $\ln s_{i|b}$ contains in part the dependent variable, and is hence endogenous. Berry recommends treating this problem with GMM, however, we will derive a likelihood expression for this, which I'm hoping might be the subject of another paper. 

Our basic approach is going to be to estimate a naive version of this equation for starting values, and then use a full-blown maximum likelihood equation by showing how the dependence filters through into the likelihood. That is, we will use the linear share form for $y_i$, but using (heuristically) a Jacobian transform to take care of the endogeneity problem duly noted by Berry; that is, if:

$$
f(y)dy=f(e)de
$$

and the distribution of $e$ is known - we shall assume it is normal with a two-way random effects structure - we can then get the distribution of $y$ - the viewership shares - through a simple transform:

$$
f(y) = f(e)\frac{de}{dy}
$$

Let's begin the process by performing some preliminary data setup jobs.


In [1]:
import ipystata
import os

CWD = os.getcwd()
print(CWD)

C:\Users\mjbaker\Documents\GitHub\NewsHour


Once again, a cut and paste job...

In [2]:
%%stata
clear all
cd C:\Users\mjbaker\Documents\GitHub\NewsHour


C:\Users\mjbaker\Documents\GitHub\NewsHour


Get our data (which we assembled in Part I) - it is an averaged data set that should be ready to use:

In [3]:
%%stata
use "Data\AveragedData.dta", clear
set more off
set seed 5150
tsset stationid timeslot


       panel variable:  stationid (strongly balanced)
        time variable:  timeslot, 3 to 8
                delta:  1 unit


## Wrangling the Data into Dynamic form (i.e., the usual panel-like setup)

One of the first things to do is get dynamic leads and lags to estimate some simple models that will somewhat resemble what we are going to do later with a complete set of random effects. One thing that is nice about doing this is that it provides somewhat of a guide as to how one might think about our more complex `Mata` code in `Stata` - we can also compare our random effects models with a more traditional two-way fixed effects model. 

Anyways, here are some lagged variables. We have lagged shares, and dummies indicating whether or not a broadcast of type $b$ in period $t$ follows a broadcast of type $b'$ in period $t-1$. We use these variables to capture the idea that people might like to watch news programs in a particular sequence over the evening.

In [4]:
%%stata
gen double lsi=l.lnsi
replace lsi=0 if lsi==.

gen lnewslnews=lnews*l.lnews
gen lnewsnnews=lnews*l.nnews
gen nnewslnews=nnews*l.lnews
gen nnewsnnews=nnews*l.nnews
replace lnewslnews=0 if lnewslnews==.
replace lnewsnnews=0 if lnewsnnews==.
replace nnewslnews=0 if nnewslnews==.
replace nnewsnnews=0 if nnewsnnews==.



(16622 missing values generated)
(16622 real changes made)
(16622 missing values generated)
(16622 missing values generated)
(16622 missing values generated)
(16622 missing values generated)
(16622 real changes made)
(16622 real changes made)
(16622 real changes made)
(16622 real changes made)


We now have interactions of shares in the current period with each of the $t,t-1$ combinations of broadcasts that we care about - i.e., what the lagged share was last period, in the event that a station broadcast, say, national news last period, and is following it with local news this period. 

In [5]:
%%stata
gen double siXlnln=lnewslnews*lsi
gen double siXlnnn=lnewsnnews*lsi
gen double siXnnln=nnewslnews*lsi
gen double siXnnnn=nnewsnnews*lsi

We also generate the total cumulative shares of local news broadcast up to a particular time slot during the night, and the total cumulative shares of national news broadcast up to a particular time slot during the night. The idea is to use these variables to capture the idea that news might get stale to later viewers because a lot of it has been watched earlier in the evening. Here goes:

In [6]:
%%stata
gen double totslnews=0
sort stationid timeslot
bysort stationid: replace totslnews=slnews[_n-1] if _n==2
bysort stationid: replace totslnews=totslnews[_n-1]+slnews[2] if _n==3
bysort stationid: replace totslnews=totslnews[_n-1]+slnews[3] if _n==4
bysort stationid: replace totslnews=totslnews[_n-1]+slnews[4] if _n==5
bysort stationid: replace totslnews=totslnews[_n-1]+slnews[5] if _n==6

gen double totsnnews=0
sort stationid timeslot
bysort stationid: replace totsnnews=snnews[_n-1] if _n==2
bysort stationid: replace totsnnews=totsnnews[_n-1]+snnews[2] if _n==3
bysort stationid: replace totsnnews=totsnnews[_n-1]+snnews[3] if _n==4
bysort stationid: replace totsnnews=totsnnews[_n-1]+snnews[4] if _n==5
bysort stationid: replace totsnnews=totsnnews[_n-1]+snnews[5] if _n==6


(16402 real changes made)
(16476 real changes made)
(16622 real changes made)
(16622 real changes made)
(16622 real changes made)
(15225 real changes made)
(16554 real changes made)
(16622 real changes made)
(16622 real changes made)
(16622 real changes made)


Because these variables can be zero, and to keep the coefficients small and stable, we will typically use a transformed version of these variables in estimation:

In [7]:
%%stata
gen double lnewstot=lnews*ln(1+totslnews)
gen double nnewstot=nnews*ln(1+totsnnews)

gen l_ACS_HH=ln(ACS_HH)

The last line gives us our log-number of households variable. 

[Ackerberg and Ryman (2006)](http://www.econ.ucla.edu/ackerber/pdfinal2.pdf) show that a multinomial logit can be really restrictive, and that it is helpful to include (functions of) group counts in estimation. They also note the role played by variation in the size of the choice set in identifying the parameters of a nested multinomial logit. Accordingly, we will include counts of each of the different types of broadcasts at given times in our estimation. These counts are easy to calculate, which can then be converted into functions as Ackerberg and Ryman (2006) suggest:

In [8]:
%%stata
bysort mt: egen total_lnews=total(lnews)
bysort mt: egen total_nnews=total(nnews)
bysort mt: egen total_otherl=total(otherl)
bysort mt: egen total_otherc=total(otherc)

gen double lnewsn=lnews*ln(1+total_lnews)
gen double nnewsn=nnews*ln(1+total_nnews)
gen double otherln=otherl*ln(1+total_otherl)
gen double othercn=otherc*ln(1+total_otherc)

## Some preliminary preliminary estimates
### The Viewership model

We can basically now estimate a proto-viewership model, which doesn't worry about endogeneity or anything, but has the basic shape as the specification we shall employ. One needs the `a2reg` package to do this, as we include market-time and station-level fixed effects in this estimation. 

Here goes (without the A/R control variables):

In [9]:
%%stata
a2reg dln ln_swg ln_swgXslnews ln_swgXsotherl ln_swgXsnnews lnews otherl nnews lnewslnews lnewsnnews nnewslnews nnewsnnews lsi siXlnln siXlnnn siXnnln siXnnnn lnewstot nnewstot l_ACS_HH, individual(stationid) unit(mt)


> stot nnewstot l_ACS_HH, individual(stationid) unit(mt)
99732 observations, 19 covariates, 16622 individuals, 1206 units, 99732 cells
Beginning Iterations
Starting Conjugate Gradient Algorithm
Iteration 0, norm of residual 4260.23108, relative error 1
Iteration 1, norm of residual 226.981112, relative error .053279061
Iteration 2, norm of residual 38.3122062, relative error .008992988
Iteration 3, norm of residual 18.9410761, relative error .004446021
Iteration 4, norm of residual 20.967133, relative error .004921595
Iteration 5, norm of residual 26.1236141, relative error .006131971
Iteration 6, norm of residual 12.3186342, relative error .002891541
Iteration 7, norm of residual 10.9890762, relative error .002579455
Iteration 8, norm of residual 6.56774712, relative error .001541641
Iteration 9, norm of residual 5.54913896, relative error .001302544
Iteration 10, norm of residual 4.10267068, relative error .000963016
Iteration 11, norm of residual 3.03920604, relative error .0007133

And here is a version with the Ackerberg/Ryman controls:

In [11]:
%%stata
a2reg dln ln_swg ln_swgXslnews ln_swgXsotherl ln_swgXsnnews lnews otherl nnews lnewslnews lnewsnnews nnewslnews nnewsnnews lsi siXlnln siXlnnn siXnnln siXnnnn lnewstot nnewstot l_ACS_HH lnewsn otherln nnewsn othercn, individual(stationid) unit(mt)


> stot nnewstot l_ACS_HH lnewsn otherln nnewsn othercn, individual(stationid) unit(mt)
99732 observations, 23 covariates, 16622 individuals, 1206 units, 99732 cells
Beginning Iterations
Starting Conjugate Gradient Algorithm
Iteration 0, norm of residual 4269.64854, relative error 1
Iteration 1, norm of residual 177.372896, relative error .041542739
Iteration 2, norm of residual 25.5208496, relative error .005977272
Iteration 3, norm of residual 16.5499861, relative error .003876194
Iteration 4, norm of residual 19.206008, relative error .004498264
Iteration 5, norm of residual 28.7209617, relative error .006726774
Iteration 6, norm of residual 7.87969834, relative error .001845515
Iteration 7, norm of residual 7.47044031, relative error .001749662
Iteration 8, norm of residual 4.9641564, relative error .001162662
Iteration 9, norm of residual 4.05031447, relative error .000948629
Iteration 10, norm of residual 2.42099193, relative error .000567024
Iteration 11, norm of residual 1.8663

The lesson from the above is that the Ackerberg/Ryman stuff seems to matter in that it changes the estimates. Note hugely, but a little bit. We therefore are going to build it into our analysis. Of course, the `a2reg` command estimates with two-way fixed, not random, effects, and does not give standard errors, but it seems like a reasonable starting place. Interestingly, it seems as though the substitability of cable programming is quite high - (.95) is the estimated value, while local programming coefficients are much lower (add the estimated .95 to the interaction term - typically on the order of -.5 or -.8 to get the ultimate result.) 

As some commenters wanted to see a pared-down model, let's estimate one of these and see where that gets us. Here, we drop all the stuff except Ackerberg/Ryman terms, dummies, and substitution parameters, but retain the two-way random effects structure:


In [12]:
%%stata
a2reg dln ln_swg ln_swgXslnews ln_swgXsotherl ln_swgXsnnews lnews otherl nnews l_ACS_HH lnewsn otherln nnewsn othercn, individual(stationid) unit(mt)


99732 observations, 12 covariates, 16622 individuals, 1206 units, 99732 cells
Beginning Iterations
Starting Conjugate Gradient Algorithm
Iteration 0, norm of residual 4269.49439, relative error 1
Iteration 1, norm of residual 177.864545, relative error .041659393
Iteration 2, norm of residual 27.9582232, relative error .006548369
Iteration 3, norm of residual 16.5501836, relative error .00387638
Iteration 4, norm of residual 19.5581728, relative error .004580911
Iteration 5, norm of residual 27.6866865, relative error .006484769
Iteration 6, norm of residual 8.67476044, relative error .0020318
Iteration 7, norm of residual 9.75644497, relative error .002285152
Iteration 8, norm of residual 5.20578669, relative error .001219298
Iteration 9, norm of residual 4.04902616, relative error .000948362
Iteration 10, norm of residual 1.95462703, relative error .000457812
Iteration 11, norm of residual 1.98984057, relative error .00046606
Iteration 12, norm of residual 3.21459096, relative error

It is somewhat amazing how little this changes things...maybe we want to go with a model like this for the ultimate product? Anyways, what if we get rid of the two-way random effects - and just estimate a linear regression (which isn't the right way to do this given the endogeneity terms, but at least gives us some hints about parameter importance and stability).

In [16]:
%%stata
reg dln ln_swg ln_swgXslnews ln_swgXsotherl ln_swgXsnnews lsi lnews otherl nnews l_ACS_HH


      Source |       SS       df       MS              Number of obs =   99732
-------------+------------------------------           F(  9, 99722) =       .
       Model |  295540.935     9  32837.8817           Prob > F      =  0.0000
    Residual |  22772.0004 99722  .228354831           R-squared     =  0.9285
-------------+------------------------------           Adj R-squared =  0.9285
       Total |  318312.936 99731  3.19171507           Root MSE      =  .47786

--------------------------------------------------------------------------------
           dln |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
---------------+----------------------------------------------------------------
        ln_swg |   .9323171   .0017398   535.87   0.000      .928907    .9357271
 ln_swgXslnews |  -.2481842   .0116391   -21.32   0.000    -.2709967   -.2253717
ln_swgXsotherl |  -.2200471   .0051013   -43.14   0.000    -.2300455   -.2100487
 ln_swgXsnnews |  -.5359728   .0062501

The plot thickens - perhaps we can get away with a very simple model...and reference people to this website if they have problems with our simple model...

But in the interests of full disclosure, note that when one includes the lagged share variable, one does not really get the expected result...(`lsi` above)

Anyways, we are first going to get everything ready to work in `Mata`, as the compiled code runs faster and also allows a bit more programming flexibility in implementation while still letting us use all Stata's great tools for graphing, displaying results, etc. We are going to use the values from `a2reg` as starting values and see if we can just run our viewership likelihood model on this. So, a first thing to do is to bring all these parameters into mata.

__Convention:__ I am going to indent mata code by four spaces, kind of like regular Python code.

We will rerun the above quietly again to get starting values (quietly)

In [25]:
%%stata
quietly a2reg dln ln_swg ln_swgXslnews ln_swgXsotherl ln_swgXsnnews lnews otherl nnews lnewslnews lnewsnnews nnewslnews nnewsnnews lsi siXlnln siXlnnn siXnnln siXnnnn lnewstot nnewstot l_ACS_HH lnewsn otherln nnewsn othercn, individual(stationid) unit(mt)


> nnn lnewstot nnewstot l_ACS_HH lnewsn otherln nnewsn othercn, individual(stationid) unit(mt)


In [26]:
%%stata
mat bDyno    =e(b)
mat sdmarDy  =e(sdunit)
mat sdstaDy  =e(sdind)
mat sdmodDy  =e(rmse)
mat alphavDy =e(constant)

mata:
    bDyno    = st_matrix("bDyno")
    sdmarDy  = st_matrix("sdmarDy")
    sdstaDy  = st_matrix("sdstaDy")
    sdmodDy  = st_matrix("sdmodDy")
    alphavDy = st_matrix("alphavDy")
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     bDyno    = st_matrix("bDyno")

:     sdmarDy  = st_matrix("sdmarDy")

:     sdstaDy  = st_matrix("sdstaDy")

:     sdmodDy  = st_matrix("sdmodDy")

:     alphavDy = st_matrix("alphavDy")

: end


Another thing that we will do is create a **structured object** to hold a bunch of variables, along with a function that initiates it, to make it easier to pass around large numbers of arguments between functions. Here is one such object, which we call `dynoInfo`. This collects all the lags and lagged shares, and a few other variables needed to fit the above model, into one thing we can hand around. 

In [27]:
%%stata
mata: 
    struct dynoInfo {
        real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews, lsi,
            siXlnln, siXlnnn, siXnnln, siXnnnn, lnewstot, nnewstot, l_ACS_HH,
            lnewsn,otherln,nnewsn,othercn
    }

    struct dynoInfo dynoInfoInit(real matrix lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
        lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,lnewsn,otherln,nnewsn,othercn)
    {
        struct dynoInfo scalar Dy
        Dy.lnewslnews=lnewslnews
        Dy.lnewsnnews=lnewsnnews
        Dy.nnewslnews=nnewslnews
        Dy.nnewsnnews=nnewsnnews
        Dy.lsi=lsi
        Dy.siXlnln=siXlnln
        Dy.siXlnnn=siXlnnn
        Dy.siXnnln=siXnnln
        Dy.siXnnnn=siXnnnn
        Dy.lnewstot=lnewstot
        Dy.nnewstot=nnewstot
        Dy.l_ACS_HH=l_ACS_HH
        Dy.lnewsn=lnewsn
        Dy.otherln=otherln
        Dy.nnewsn=nnewsn
        Dy.othercn=othercn
        return(Dy)
    }
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     struct dynoInfo {
dynoInfo() already exists
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
r(3000);
unrecognized command:  real
r(199);
unrecognized command:  siXlnln
r(199);
unrecognized command:  lnewsn
r(199);
unrecognized command:  } invalid command name
r(199);
unrecognized command:  struct
r(199);
unrecognized command:  lsi
r(199);
.         struct dynoInfo scalar Dy
unrecognized command:  struct
r(199);.         Dy.lnewsnnews=lnewsnnews.         Dy.nnewsnnews=nnewsnnews.         Dy.siXlnln=siXlnln.         Dy.siXnnln=siXnnln.         Dy.lnewstot=lnewstot.         Dy.l_ACS_HH=l_ACS_HH.         Dy.otherln=otherln.         Dy.othercn=othercn.     }
r(199);
unrecognized command:  end
r(199);


Before going to much furher, we have to set up and specify a viewership likelihood in mata. I should more thoroughly describe what is going on here, and I will, but for now I will note that the likelihood function is essentially computed as is done in [Baltagi's book](http://www.wiley.com/WileyCDA/WileyTitle/productCd-EHEP003191.html), with the rather large exception of adding on a log-determinant adjustment for the endogeneity. I will add a description here as to where this comes from.

In [19]:
%%stata
mata:
    void logLikelihood2DynaA(M,todo,b,lnf,g,H)
    {
        real scalar mu_l,mu_o,mu_n,mu_c,eta_l,eta_n,
            sdsta,sdmar,sdmod,i,T,N,lam1,
            lam2,lam3,lam4,phi22,phi32
        real matrix X,mt,id,mtp,idp,yp,Xp,Jn,Jt,En,Et,
            Q1,Q2,Q3,Q4,OmegaInv,beta
        struct dynoInfo scalar Dy

        mu_l =(moptimize_util_xb(M,b,1))^2
        mu_o =(moptimize_util_xb(M,b,2))^2
        mu_n =(moptimize_util_xb(M,b,3))^2
        mu_c =(moptimize_util_xb(M,b,4))^2
        eta_l=moptimize_util_xb(M,b,5)
        eta_o=moptimize_util_xb(M,b,6)
        eta_n=moptimize_util_xb(M,b,7)
        eta_ll=moptimize_util_xb(M,b,8)
        eta_ln=moptimize_util_xb(M,b,9)
        eta_nl=moptimize_util_xb(M,b,10)
        eta_nn=moptimize_util_xb(M,b,11)

        lam_own=moptimize_util_xb(M,b,12)
        lam_ll=moptimize_util_xb(M,b,13)
        lam_ln=moptimize_util_xb(M,b,14)

        lam_nl=moptimize_util_xb(M,b,15)
        lam_nn=moptimize_util_xb(M,b,16)
        rho_l=moptimize_util_xb(M,b,17)
        rho_n=moptimize_util_xb(M,b,18)
        omega=moptimize_util_xb(M,b,19)
        zeta_l=moptimize_util_xb(M,b,20)
        zeta_o=moptimize_util_xb(M,b,21)
        zeta_n=moptimize_util_xb(M,b,22)
        zeta_c=moptimize_util_xb(M,b,23)
        sdsta=exp(moptimize_util_xb(M,b,24))
        sdmar=exp(moptimize_util_xb(M,b,25))
        sdmod=exp(moptimize_util_xb(M,b,26))

        alpha=moptimize_util_xb(M,b,27)

        y     =moptimize_util_depvar(M,1)
        lnswg =moptimize_util_depvar(M,2)
        lnews =moptimize_util_depvar(M,3)
        otherl=moptimize_util_depvar(M,4)
        nnews =moptimize_util_depvar(M,5)
        otherc=moptimize_util_depvar(M,6)
        l_ACS_HH=moptimize_util_depvar(M,7)

        id=moptimize_util_userinfo(M,1)
        mt=moptimize_util_userinfo(M,2)
        m =moptimize_util_userinfo(M,3)
        Dy =moptimize_util_userinfo(M,4)
        si =moptimize_util_userinfo(M,5)

        beta=mu_l,mu_o,mu_n,mu_c,eta_l,eta_o,eta_n,
            eta_ll,eta_ln,eta_nl,eta_nn,lam_own,lam_ll,
            lam_ln,lam_nl,lam_nn,rho_l,rho_n,omega,zeta_l,zeta_o,zeta_n,zeta_c,alpha

        X=lnews:*lnswg,otherl:*lnswg,nnews:*lnswg,otherc:*lnswg,
            lnews,otherl,nnews,
            Dy.lnewslnews,Dy.lnewsnnews,Dy.nnewslnews,Dy.nnewsnnews,
            Dy.lsi,Dy.siXlnln,Dy.siXlnnn,Dy.siXnnln,Dy.siXnnnn,
            Dy.lnewstot,Dy.nnewstot,l_ACS_HH,Dy.lnewsn,Dy.otherln,Dy.nnewsn,Dy.othercn,J(rows(lnews),1,1)

        B=lnews,otherl,nnews,otherc

        lnf=J(rows(m),1,.)
        lnDetF=J(rows(m),6,.)
        muVec=mu_l,mu_o,mu_n,mu_c
        if (runiform(1,1)>.96) muVec,eta_l,eta_n,omega,zeta_l,zeta_o,zeta_n,zeta_c
            for (i=1;i<=rows(m);i++) {
                mtp=panelsubmatrix(mt,i,m)
                idp=panelsubmatrix(id,i,m)
                T=rows(uniqrows(mtp))
                N=rows(uniqrows(idp))
                yp=panelsubmatrix(y,i,m)
                Xp=panelsubmatrix(X,i,m)
                sip=panelsubmatrix(si,i,m)
                Bp=panelsubmatrix(B,i,m)
            
                lam1=sdmod^2
                lam2=T*sdsta^2+sdmod^2
                lam3=N*sdmar^2+sdmod^2
                lam4=T*sdsta^2+N*sdmar^2+sdmod^2
                phi22=sdmod^2/lam2
                phi32=sdmod^2/lam3
                phi42=sdmod^2/lam4

                Jn=J(N,N,1/N)
                Jt=J(T,T,1/T)
                En=I(N)-Jn
                Et=I(T)-Jt

                Q1=En#Et
                Q2=En#Jt
                Q3=Jn#Et
                Q4=Jn#Jt
                OmegaInv=(Q1/lam1+Q2/lam2+Q3/lam3+Q4/lam4)
                lnDetOmega=-2*N*T*ln(sdmod)+(N-1)*ln(phi22)+(T-1)*ln(phi32)+ln(phi42)

                lnf[i]=1/2*lnDetOmega-1/2*(yp-Xp*beta')'*OmegaInv*(yp-Xp*beta')

                TimeVars=uniqrows(mtp)	

                for (z=1;z<=rows(TimeVars);z++) {
                    siz=select(sip,mtp:==TimeVars[z])
                    Bz=select(Bp,mtp:==TimeVars[z])
                    sizBz=siz:*Bz
                    sg=colsum(sizBz)
                    Ng=colsum(sizBz:!=0)
                    term=(Ng:>0):*(Ng:-1):*ln(1:-muVec)
                    if (hasmissing(term)) lnDetF[i,z]=.
                    else lnDetF[i,z]=rowsum(term)
                }
            }

        lnDetF=B:*ln(1:-muVec)
        if (hasmissing(lnf)) lnf=.
        else lnf=colsum(lnf)
        if (hasmissing(lnDetF)) lnf=.
        else lnf=colsum(lnf) +colsum(rowsum(lnDetF))
        if (runiform(1,1)>.96) colsum(rowsum(lnDetF)),lnf
    }
end                                                                
                                                                   


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     void logLikelihood2DynaA(M,todo,b,lnf,g,H)
>     {
>         real scalar mu_l,mu_o,mu_n,mu_c,eta_l,eta_n,
>             sdsta,sdmar,sdmod,i,T,N,lam1,
>             lam2,lam3,lam4,phi22,phi32
>         real matrix X,mt,id,mtp,idp,yp,Xp,Jn,Jt,En,Et,
>             Q1,Q2,Q3,Q4,OmegaInv,beta
>         struct dynoInfo scalar Dy
> 
>         mu_l =(moptimize_util_xb(M,b,1))^2
>         mu_o =(moptimize_util_xb(M,b,2))^2
>         mu_n =(moptimize_util_xb(M,b,3))^2
>         mu_c =(moptimize_util_xb(M,b,4))^2
>         eta_l=moptimize_util_xb(M,b,5)
>         eta_o=moptimize_util_xb(M,b,6)
>         eta_n=moptimize_util_xb(M,b,7)
>         eta_ll=moptimize_util_xb(M,b,8)
>         eta_ln=moptimize_util_xb(M,b,9)
>         eta_nl=moptimize_util_xb(M,b,10)
>         eta_nn=moptimize_util_xb(M,b,11)
> 
>         lam_own=moptim

Make sure the data is sorted correctly (yet again - one can't be too careful about this!) and then pull in market, station id, and market-time identifiers. We also need to adjust parameters for our model because when using a linear estimation routine like `a2reg`, we use interactions for within-group shares. But when doing maximum likelihood, we will specify these parameters as-is, hence we have to back out the values by adding parameters and interactions: 

In [28]:
%%stata

sort market stationid mt

mata: 
    st_view(ma=.,.,"market")
    st_view(id=.,.,"stationid")
    st_view(mt=.,.,"mt")
    
    boDyno=bDyno
    boDyno[1]=bDyno[1]+bDyno[2]
    boDyno[2]=bDyno[1]+bDyno[3]
    boDyno[3]=bDyno[1]+bDyno[4]
    boDyno[4]=bDyno[1]
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     st_view(ma=.,.,"market")

:     st_view(id=.,.,"stationid")

:     st_view(mt=.,.,"mt")

:     
:     boDyno=bDyno

:     boDyno[1]=bDyno[1]+bDyno[2]

:     boDyno[2]=bDyno[1]+bDyno[3]

:     boDyno[3]=bDyno[1]+bDyno[4]

:     boDyno[4]=bDyno[1]

: end


Now, here we can begin the process of:

1. Setting up an `optimize` object in mata, 
2. Passing it initial values for parameters
3. Passing it all variables from the model. 

This also serves as a way of mapping variables to the variable names we use in the paper.

In [29]:
%%stata
mata: 
    Z=moptimize_init()
    moptimize_init_trace_dots(Z,"on")
    moptimize_init_trace_params(Z,"on")
    moptimize_init_evaluator(Z,&logLikelihood2DynaA())   
    moptimize_init_evaluatortype(Z,"d0")    
    moptimize_init_which(Z,"max")

    moptimize_init_eq_indepvars(Z,1,"")
    moptimize_init_eq_indepvars(Z,2,"")
    moptimize_init_eq_indepvars(Z,3,"")
    moptimize_init_eq_indepvars(Z,4,"")
    moptimize_init_eq_indepvars(Z,5,"")
    moptimize_init_eq_indepvars(Z,6,"")
    moptimize_init_eq_indepvars(Z,7,"")
    
    moptimize_init_eq_indepvars(Z,8,"")
    moptimize_init_eq_indepvars(Z,9,"")
    moptimize_init_eq_indepvars(Z,10,"")
    moptimize_init_eq_indepvars(Z,11,"")

    moptimize_init_eq_indepvars(Z,12,"")
    moptimize_init_eq_indepvars(Z,13,"")
    moptimize_init_eq_indepvars(Z,14,"")
    moptimize_init_eq_indepvars(Z,15,"")
    moptimize_init_eq_indepvars(Z,16,"")

    moptimize_init_eq_indepvars(Z,17,"") 
    moptimize_init_eq_indepvars(Z,18,"")  
    moptimize_init_eq_indepvars(Z,19,"")  

    moptimize_init_eq_indepvars(Z,20,"")
    moptimize_init_eq_indepvars(Z,21,"")
    moptimize_init_eq_indepvars(Z,22,"")
    moptimize_init_eq_indepvars(Z,23,"")
    

    moptimize_init_eq_indepvars(Z,24,"") 
    moptimize_init_eq_indepvars(Z,25,"")  
    moptimize_init_eq_indepvars(Z,26,"")  
    moptimize_init_eq_indepvars(Z,27,"")  

    moptimize_init_depvar(Z,1,"dln")
    moptimize_init_depvar(Z,2,"ln_swg")
    moptimize_init_depvar(Z,3,"lnews")
    moptimize_init_depvar(Z,4,"otherl")
    moptimize_init_depvar(Z,5,"nnews")
    moptimize_init_depvar(Z,6,"otherc")
    moptimize_init_depvar(Z,7,"l_ACS_HH")


    st_view(lnewslnews=.,.,"lnewslnews")
    st_view(lnewsnnews=.,.,"lnewsnnews")
    st_view(nnewslnews=.,.,"nnewslnews")
    st_view(nnewsnnews=.,.,"nnewsnnews")
    st_view(siXlnln=.,.,"siXlnln")
    st_view(siXlnnn=.,.,"siXlnnn")
    st_view(siXnnln=.,.,"siXnnln")
    st_view(siXnnnn=.,.,"siXnnnn")
    st_view(lnewstot=.,.,"lnewstot")
    st_view(nnewstot=.,.,"nnewstot")
    st_view(si=.,.,"si")
    st_view(lsi=.,.,"lsi")
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(othercn=.,.,"othercn")
    
    Dy=dynoInfoInit(lnewslnews, lnewsnnews, nnewslnews, nnewsnnews,
        lsi,siXlnln, siXlnnn,siXnnln, siXnnnn, lnewstot, nnewstot,l_ACS_HH,
        lnewsn,otherln,nnewsn,othercn)

    m=panelsetup(ma,1)
    moptimize_init_userinfo(Z,1,id)
    moptimize_init_userinfo(Z,2,mt)
    moptimize_init_userinfo(Z,3,m)
    moptimize_init_userinfo(Z,4,Dy)
    moptimize_init_userinfo(Z,5,si)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     Z=moptimize_init()

:     moptimize_init_trace_dots(Z,"on")

:     moptimize_init_trace_params(Z,"on")

:     moptimize_init_evaluator(Z,&logLikelihood2DynaA())   

:     moptimize_init_evaluatortype(Z,"d0")    

:     moptimize_init_which(Z,"max")

:     moptimize_init_eq_indepvars(Z,2,"")

:     moptimize_init_eq_indepvars(Z,3,"")

:     moptimize_init_eq_indepvars(Z,4,"")

:     moptimize_init_eq_indepvars(Z,5,"")

:     moptimize_init_eq_indepvars(Z,6,"")

:     moptimize_init_eq_indepvars(Z,7,"")

:     
:     moptimize_init_eq_indepvars(Z,8,"")

:     moptimize_init_eq_indepvars(Z,9,"")

:     moptimize_init_eq_indepvars(Z,10,"")

:     moptimize_init_eq_indepvars(Z,11,"")

:     moptimize_init_eq_indepvars(Z,13,"")

:     moptimize_init_eq_indepvars(Z,14,"")

:     moptimize_init_eq_indepvars(Z,15,"")

:     

Just a test to see if it is working...

In [23]:
%%stata
mata: moptimize_evaluate(Z)

Now, we set up things to do an `AMCMC` run on the above, to see where everything winds up going...We also need to arrange starting values for parameters, which we do according to the estimated `a2reg` model, where we tack on standard deviations for other parameters. Setting up such a run can be read about in [my paper here](https://ideas.repec.org/a/tsj/stataj/v14y2014i3p623-661.html).

Note we will use these as starting values in the full model, so we save those results as well. Be sure to include `.mmat` files in the `.gitignore` file!

In [46]:
%%stata
mata: 
    bo=boDyno[1..23],sdmarDy,sdstaDy,sdmodDy,alphavDy
    alginfo="mwg","d0","moptimize"
    b_start=amcmc(alginfo,&logLikelihood2DynaA(),bo,diag(abs(bo)/2),201,101,1,.4,arate=.,vals=.,lambda=.,.,Z)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     bo=boDyno[1..23],sdmarDy,sdstaDy,sdmodDy,alphavDy

:     alginfo="mwg","d0","moptimize"

:     b_start=amcmc(alginfo,&logLikelihood2DynaA(),bo,diag(abs(bo)/2),201,101,1,.4,arate=.,vals=.,lambda=.,.,Z)
                  1              2
    +-------------------------------+
  1 |  -2127.664941              .  |
    +-------------------------------+
                 1             2             3             4             5             6             7             8             9            10            11
    +-----------------------------------------------------------------------------------------------------------------------------------------------------------+
  1 |   .213297921   .2184523592     .05173213   .9426971907    4.81550257    3.65982098   -.525220594    1.10998523    1.32053662     .85103021    2.152107

We will now save all these variables so we don't have to keep doing this - they can be good start values for what follows. I will save the last draw (so we have a starting value for a vector), but also the likelihood values, the acceptance rate from the draws, and all the draws themselves just so we have them if needed. 

In [47]:
%%stata
mata: 
    bo=b_start[rows(b_start),]
    drawsbo=b_start
    mata matsave DynoStarts23 bo drawsbo vals arate, replace
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     bo=b_start[rows(b_start),]

:     drawsbo=b_start

:     mata matsave DynoStarts23 bo drawsbo vals arate, replace
(saving arate[1,27], bo[1,27], drawsbo[100,27], vals[100,1])
file DynoStarts23.mmat saved

: end


## A check of adding up and prediction...

One thing to do before going much further is to see if predictions from the model - replete with error terms - actually match up with the shares that we have. Let's take a look. We can do this independently of the estimation above because we saved the results. Anyways:


In [51]:
%%stata
mata:
    mata matuse DynoStarts23, replace
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     mata matuse DynoStarts23, replace
(loading arate[1,27], bo[1,27], drawsbo[100,27], vals[100,1])

: end


A first step is to recreate our parameters (the ones that have been transformed, anyways). Next, we have to generate linear predictions, reshape variables, and run them through the model. Along the way, we also have to add in a Mata function that computes shares. 

**Note:** The `xBpUE` variable is in fact the entire linear prediction:

$$
\hat{U}=X\hat{\beta}+\hat{\epsilon}
$$

without the share parameters. Hence, if one has $\hat{U}$ and the share parameters, one should be able 

In [52]:
%%stata
mata:
    bo[,1::4]=bo[,1::4]:^2
    bo=bo[rows(bo),]
    
    st_view(ln_swg=.,.,"ln_swg")
    st_view(lnews=.,.,"lnews")
    st_view(otherl=.,.,"otherl")
    st_view(nnews=.,.,"nnews")
    st_view(otherc=.,.,"otherc")
    st_view(dln=.,.,"dln")

    xBpUE=dln:-  bo[1]:*ln_swg:*lnews:-
         bo[2]:*ln_swg:*otherl:-
         bo[3]:*ln_swg:*nnews:-
         bo[4]:*ln_swg:*otherc
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     bo[,1::4]=bo[,1::4]:^2

:     bo=bo[rows(bo),]

:     
:     st_view(ln_swg=.,.,"ln_swg")

:     st_view(lnews=.,.,"lnews")

:     st_view(otherl=.,.,"otherl")

:     st_view(nnews=.,.,"nnews")

:     st_view(otherc=.,.,"otherc")

:     st_view(dln=.,.,"dln")
>          bo[2]:*ln_swg:*otherl:-
>          bo[3]:*ln_swg:*nnews:-
>          bo[4]:*ln_swg:*otherc

: end


Next, I reshape data so that it is in `wide` form (which for some reason I called `Long` as a suffix in the data. 

In [53]:
%%stata
mata:
    timeslots=6
    lnewsLong=lnews
    otherlLong=otherl
    nnewsLong=nnews
    othercLong=otherc
    
    lnewsLong=colshape(lnewsLong,6)
    otherlLong=colshape(otherlLong,6)
    nnewsLong=colshape(nnewsLong,6)
    othercLong=colshape(othercLong,6)
    xBpUE=colshape(xBpUE,6)

    simSharesLong=J(rows(lnewsLong),cols(lnewsLong),.)
    simSharesLong2=J(rows(lnewsLong),cols(lnewsLong),.)
    simSharesLong3=J(rows(lnewsLong),cols(lnewsLong),.)
    simSharesLong4=J(rows(lnewsLong),cols(lnewsLong),.)
    marketIdLong=ma
    marketIdLong=colshape(marketIdLong,6)
    mLong=panelsetup(marketIdLong,1)
end    


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     timeslots=6

:     lnewsLong=lnews

:     otherlLong=otherl

:     nnewsLong=nnews

:     othercLong=otherc

:     
:     lnewsLong=colshape(lnewsLong,6)

:     otherlLong=colshape(otherlLong,6)

:     nnewsLong=colshape(nnewsLong,6)

:     othercLong=colshape(othercLong,6)

:     xBpUE=colshape(xBpUE,6)

:     simSharesLong2=J(rows(lnewsLong),cols(lnewsLong),.)

:     simSharesLong3=J(rows(lnewsLong),cols(lnewsLong),.)

:     simSharesLong4=J(rows(lnewsLong),cols(lnewsLong),.)

:     marketIdLong=ma

:     marketIdLong=colshape(marketIdLong,6)

:     mLong=panelsetup(marketIdLong,1)

: end


## Computing shares given linear index values.

The next thing to be done is to create some means of simulating shares, given linera index values. We will try this with several different methods, just to be safe, and to get a feel for if underflow problems are present, and if so, which method tends to do the best job with them. Anyways, here are some functions that calculate shares:

In [54]:
%%stata
mata:
    real matrix eshares_up(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
    {
        sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
        lu=exp(Uv:/(1:-sigg))
        lu_l=colsum(lnews:*lu)
        lu_n=colsum(nnews:*lu)
        lu_o=colsum(otherl:*lu)
        lu_c=colsum(otherc:*lu)
        swg=lu:/(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc)
        share=swg:*(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc):^(1:-sigg):/
                  (1:+lu_l:^(1:-sigma_l):+lu_n:^(1:-sigma_n):+lu_o:^(1:-sigma_o):+lu_c:^(1:-sigma_c))
        return(share)
    }
    
    real matrix esharesStable(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
    {
        sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
        lnLu=Uv:/(1:-sigg)
        if (colsum(lnews)>0) {
            lnU=select(lnLu,lnews)
            max=max(lnU)
            lu_l=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_l=0
        if (colsum(nnews)>0) {
            lnU=select(lnLu,nnews)
            max=max(lnU)
            lu_n=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_n=0
        if (colsum(otherl)>0) {
            lnU=select(lnLu,otherl)
            max=max(lnU)
            lu_o=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_o=0
        if (colsum(otherc)>0) {
            lnU=select(lnLu,otherc)
            max=max(lnU)
            lu_c=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_c=0
        swg=exp(lnLu):/(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc)
        share=swg:*(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc):^(1:-sigg):/
              (1:+lu_l:^(1:-sigma_l):+lu_n:^(1:-sigma_n):+lu_o:^(1:-sigma_o):+lu_c:^(1:-sigma_c))	
        return(share)
    }
    
    real matrix esharesMoreStable(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
    {
        sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
        lnLu=Uv:/(1:-sigg)
        if (colsum(lnews)>0) {
            lnU=select(lnLu,lnews)
            max=max(lnU)
            lu_l=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_l=0
        if (colsum(nnews)>0) {
            lnU=select(lnLu,nnews)
            max=max(lnU)
            lu_n=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_n=0
        if (colsum(otherl)>0) {
            lnU=select(lnLu,otherl)
            max=max(lnU)
            lu_o=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_o=0
        if (colsum(otherc)>0) {
            lnU=select(lnLu,otherc)
            max=max(lnU)
            lu_c=exp(max):*colsum(exp(lnU:-max))
        }
        else lu_c=0
        lso=ln(1:+lu_l:^(1-sigma_l):+lu_o:^(1-sigma_o):+lu_n:^(1-sigma_n):+lu_c:^(1-sigma_c))
        lnlu_l=ln(lu_l)
        lnlu_o=ln(lu_o)
        lnlu_n=ln(lu_n)
        lnlu_c=ln(lu_c)
        _editmissing(lnlu_l,0)
        _editmissing(lnlu_o,0)
        _editmissing(lnlu_c,0)
        _editmissing(lnlu_n,0)
        lsi=lnLu:-lnlu_l:*sigma_l:*lnews:-lnlu_n:*sigma_n:*nnews:-lnlu_o:*sigma_o:*otherl:-lnlu_c:*sigma_c:*otherc:-lso
        return(exp(lsi))
    }

    real matrix esharesQuadPres(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
    {
        sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
        lnLu=Uv:/(1:-sigg)
        if (colsum(lnews)>0) {
            lnU=select(lnLu,lnews)
            max=max(lnU)
            lu_l=exp(max):*quadcolsum(exp(lnU:-max))
        }
        else lu_l=0
        if (colsum(nnews)>0) {
            lnU=select(lnLu,nnews)
            max=max(lnU)
            lu_n=exp(max):*quadcolsum(exp(lnU:-max))
        }
        else lu_n=0
        if (colsum(otherl)>0) {
            lnU=select(lnLu,otherl)
            max=max(lnU)
            lu_o=exp(max):*quadcolsum(exp(lnU:-max))
        }
        else lu_o=0
        if (colsum(otherc)>0) {
            lnU=select(lnLu,otherc)
            max=max(lnU)
            lu_c=exp(max):*quadcolsum(exp(lnU:-max))
        }
        else lu_c=0
        lso=ln(1:+lu_l:^(1-sigma_l):+lu_o:^(1-sigma_o):+lu_n:^(1-sigma_n):+lu_c:^(1-sigma_c))
        lnlu_l=ln(lu_l)
        lnlu_o=ln(lu_o)
        lnlu_n=ln(lu_n)
        lnlu_c=ln(lu_c)
        _editmissing(lnlu_l,0)
        _editmissing(lnlu_o,0)
        _editmissing(lnlu_c,0)
        _editmissing(lnlu_n,0)
        lsi=lnLu:-lnlu_l:*sigma_l:*lnews:-lnlu_n:*sigma_n:*nnews:-lnlu_o:*sigma_o:*otherl:-lnlu_c:*sigma_c:*otherc:-lso
        return(exp(lsi))
    }
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     real matrix eshares_up(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
>     {
>         sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
>         lu=exp(Uv:/(1:-sigg))
>         lu_l=colsum(lnews:*lu)
>         lu_n=colsum(nnews:*lu)
>         lu_o=colsum(otherl:*lu)
>         lu_c=colsum(otherc:*lu)
>         swg=lu:/(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc)
>         share=swg:*(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc):^(1:-sigg):/
>                   (1:+lu_l:^(1:-sigma_l):+lu_n:^(1:-sigma_n):+lu_o:^(1:-sigma_o):+lu_c:^(1:-sigma_c))
>         return(share)
>     }

:     
:     real matrix esharesStable(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
>     {
>         sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c


The functions really only differ in that they get progressively more careful about the handling of zeros. Note that the "underflow" trick is applied. Instead of calculating:

$$
\ln(\sum e^x_i)
$$

I use in some of the functions:

$$
M+\ln(\sum(e^{x_i-M})
$$

Where $M$ is the maximum value of the $x_i$'s. This turns out to be a pretty stable way of doing things, as one at least has a one term inside the log. 

Having set up shares functions, placeholders, linear indexes, and all that, we can now put together shares through the following simple loop. For comparison, we can also reshape the original shares

In [55]:
%%stata
mata:
    for (i=1;i<=rows(mLong);i++){
        xBpUEp=panelsubmatrix(xBpUE,i,mLong)
        lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
        otherlLongp=panelsubmatrix(otherlLong,i,mLong)
        nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
        othercLongp=panelsubmatrix(othercLong,i,mLong)
        for (t=1;t<=timeslots;t++) {
            simSharesLong[mLong[i,1]::mLong[i,2],t]=
                esharesStable(xBpUEp[,t],lnewsLongp[,t],otherlLongp[,t],
                nnewsLongp[,t],othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
            simSharesLong2[mLong[i,1]::mLong[i,2],t]=
                eshares_up(xBpUEp[,t],lnewsLongp[,t],otherlLongp[,t],
                nnewsLongp[,t],othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
            simSharesLong3[mLong[i,1]::mLong[i,2],t]=
                esharesMoreStable(xBpUEp[,t],lnewsLongp[,t],otherlLongp[,t],
                nnewsLongp[,t],othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
            simSharesLong4[mLong[i,1]::mLong[i,2],t]=
                esharesQuadPres(xBpUEp[,t],lnewsLongp[,t],otherlLongp[,t],
                nnewsLongp[,t],othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
        }
    }

    siLong=si
    siLong=colshape(siLong,6)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     for (i=1;i<=rows(mLong);i++){
>         xBpUEp=panelsubmatrix(xBpUE,i,mLong)
>         lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
>         otherlLongp=panelsubmatrix(otherlLong,i,mLong)
>         nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
>         othercLongp=panelsubmatrix(othercLong,i,mLong)
>         for (t=1;t<=timeslots;t++) {
>             simSharesLong[mLong[i,1]::mLong[i,2],t]=
>                 esharesStable(xBpUEp[,t],lnewsLongp[,t],otherlLongp[,t],
>                 nnewsLongp[,t],othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
>             simSharesLong2[mLong[i,1]::mLong[i,2],t]=
>                 eshares_up(xBpUEp[,t],lnewsLongp[,t],otherlLongp[,t],
>                 nnewsLongp[,t],othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
>             simSharesLong3[mLong[i,1]::mLong[i,2],t]=
>                 eshares

In [58]:
%%stata
    mata: max(abs(simSharesLong:-siLong))
    mata: max(abs(simSharesLong2:-siLong))
    mata: max(abs(simSharesLong3:-siLong))
    mata: max(abs(simSharesLong4:-siLong))


  8.32667e-17
  1.11022e-16
  8.32667e-17
  8.32667e-17


This suggests we can breathe easy and adopt a simple approach to computing shares. We will, however, rely mainly on the `esharesStable` approach just to be safe! We see that there is little gain in doing things with quad precision, as everything is done that way in mata anyways.

## Save data and move onto the next part

In [60]:
%%stata
save AveragedDataDyno.dta, replace


(note: file AveragedDataDyno.dta not found)
file AveragedDataDyno.dta saved
